In [57]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from wrangle import get_zillow

In [58]:
df = get_zillow()

In [59]:
df.head()

,Unnamed: 0,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
0,0,0.0,0.0,NaN,9.0,NaN,NaN,6037.0
1,1,0.0,0.0,NaN,27516.0,NaN,NaN,6037.0
2,2,0.0,0.0,73026.0,1434941.0,1959.0,20800.37,6037.0
3,3,0.0,0.0,5068.0,1174475.0,1948.0,14557.57,6037.0
4,4,0.0,0.0,1776.0,440101.0,1947.0,5725.17,6037.0


In [60]:
df.shape

(2985217, 8)

In [61]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Unnamed: 0,2985217.0,1.492608e+06,861758.063609,0.00,746304.00,1492608.00,2238912.0,2.985216e+06
bedroomcnt,2982272.0,3.093362e+00,1.271569,0.00,2.00,3.00,4.0,2.500000e+01
bathroomcnt,2982260.0,2.216196e+00,1.078250,0.00,2.00,2.00,3.0,3.200000e+01
calculatedfinishedsquarefeet,2940120.0,1.831455e+03,1954.198051,1.00,1215.00,1574.00,2140.0,9.525760e+05
taxvaluedollarcnt,2950951.0,4.435279e+05,816336.627054,1.00,188220.00,321161.00,514072.0,3.196225e+08
yearbuilt,2937384.0,1.964438e+03,23.644275,1801.00,1950.00,1963.00,1981.0,2.016000e+03
taxamount,2962465.0,5.408949e+03,9675.566706,0.24,2468.62,4007.62,6230.5,3.823176e+06
fips,2982285.0,6.048053e+03,20.240825,6037.00,6037.00,6037.00,6059.0,6.111000e+03


In [62]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2985217 entries, 0 to 2985216
Data columns (total 8 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   Unnamed: 0                    int64  
 1   bedroomcnt                    float64
 2   bathroomcnt                   float64
 3   calculatedfinishedsquarefeet  float64
 4   taxvaluedollarcnt             float64
 5   yearbuilt                     float64
 6   taxamount                     float64
 7   fips                          float64
dtypes: float64(7), int64(1)
memory usage: 182.2 MB


In [63]:
# This data is very disqusting. Why is bedroom a float... what is 0.5 of a bedroom? Harry Potter Under the Stairs?
# Same with Year Built, why is it a float? It's either one year or another, I'm not looking for the month, what would December even be 1985.92 ??

In [64]:
df.isnull().sum()

Unnamed: 0                          0
bedroomcnt                       2945
bathroomcnt                      2957
calculatedfinishedsquarefeet    45097
taxvaluedollarcnt               34266
yearbuilt                       47833
taxamount                       22752
fips                             2932
dtype: int64

In [65]:
df.isnull().any()

Unnamed: 0                      False
bedroomcnt                       True
bathroomcnt                      True
calculatedfinishedsquarefeet     True
taxvaluedollarcnt                True
yearbuilt                        True
taxamount                        True
fips                             True
dtype: bool

In [66]:
df.shape

(2985217, 8)

In [67]:
# There is almost 3 million entries. I'm willing to miss a few.
df.dropna().info()
# This will drop 83,299 entries, how much of my data would I actually drop?
83299/2985217
# Just under 3% that might be more than I'm willing. I'll test out different cleaning methods.

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2901918 entries, 2 to 2982283
Data columns (total 8 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   Unnamed: 0                    int64  
 1   bedroomcnt                    float64
 2   bathroomcnt                   float64
 3   calculatedfinishedsquarefeet  float64
 4   taxvaluedollarcnt             float64
 5   yearbuilt                     float64
 6   taxamount                     float64
 7   fips                          float64
dtypes: float64(7), int64(1)
memory usage: 199.3 MB


0.02790383412663133

In [68]:
df.isnull().sum()

Unnamed: 0                          0
bedroomcnt                       2945
bathroomcnt                      2957
calculatedfinishedsquarefeet    45097
taxvaluedollarcnt               34266
yearbuilt                       47833
taxamount                       22752
fips                             2932
dtype: int64

In [76]:
# I'm going to fill in the least missing, so bedroomcnt and bathroomcnt, 
# I don't know what fips is, so I'm not going to mess with it incase it scews the data somehow.
df.fips.nunique(), df.fips.unique()
# Yeah, fips is something very specific so I'm not messing with that.

(3, array([6037., 6059., 6111.,   nan]))

In [69]:
df.bedroomcnt.mean(), df.bedroomcnt.mode(), df.bedroomcnt.median()

(3.093362040752822,
 0    3.0
 Name: bedroomcnt, dtype: float64,
 3.0)

In [70]:
# Using median because they're similar, I'd have to round the mean anyways to get a whole number for turning into an int.

df.bedroomcnt = df.bedroomcnt.replace(np.nan, df.bedroomcnt.median())
df.bedroomcnt.isnull().sum()

0

In [71]:
df.bathroomcnt.mean(), df.bathroomcnt.mode(), df.bathroomcnt.median()

(2.216195770992469,
 0    2.0
 Name: bathroomcnt, dtype: float64,
 2.0)

In [72]:
# Just checking they are doing half bath even though it will still round to 2.
df.bathroomcnt.unique()

array([ 0.  ,  4.  ,  2.  ,  3.  ,  1.  ,  2.5 ,  3.5 ,  5.5 ,  4.5 ,
        5.  ,  1.5 ,  7.5 ,  6.  ,  7.  , 10.  ,  8.  ,  9.  , 12.  ,
       11.  ,  8.5 ,  6.5 , 13.  ,  9.5 , 14.  , 20.  , 19.5 , 10.5 ,
         nan, 15.  , 18.  , 16.  ,  1.75, 17.  , 19.  ,  0.5 , 12.5 ,
       11.5 , 14.5 , 32.  , 31.  ])

In [73]:
# Mean and mode round the same, so I'm just using mode for no rounding.
df.bathroomcnt = df.bathroomcnt.replace(np.nan, df.bathroomcnt.median())
df.bathroomcnt.isnull().sum()

0

-----

In [75]:
# bedroomcnt needs to be a whole number, half bath is a thing, half bed isn't.
# I'll throw in year built since it can't be a float either.
# Looking at calc sqft and taxs in dollars, they should be whole numbers too since they're already rounded before entry.

In [ ]:
df.bedroomcnt.astype(int)
df.calculatedfinishedsquarefeet.astype(int)
df.taxvaluedollarcnt.astype(int)
df.yearbuilt.astype(int)